In [ ]:
!pip install gdown
import gdown

In [ ]:
file_id = '1XqTGbpH7OK6cqUIP2_O5JFIUihSQAUND'  # ganti dengan ID milikmu
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'dataset_final_split.zip', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1XqTGbpH7OK6cqUIP2_O5JFIUihSQAUND
From (redirected): https://drive.google.com/uc?id=1XqTGbpH7OK6cqUIP2_O5JFIUihSQAUND&confirm=t&uuid=5c0a0c3b-ab34-4636-95db-de4730c2dd4a
To: /content/dataset_final_split.zip
100%|██████████| 230M/230M [00:01<00:00, 161MB/s]


'dataset_final_split.zip'

In [ ]:
import os
import zipfile

In [ ]:
with zipfile.ZipFile('dataset_final_split.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [ ]:
import shutil

# Pindahkan folder
shutil.move('dataset/content/data_final_split', 'data_final_split')

# (Opsional) Hapus folder 'dataset/content'
shutil.rmtree('dataset')

In [ ]:
base_dir = 'data_final_split'
splits = ['train', 'val', 'test']

for split in splits:
    print(f"\n=== {split.upper()} ===")
    split_dir = os.path.join(base_dir, split)
    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        if os.path.isdir(class_dir):
            n_files = len(os.listdir(class_dir))
            print(f"{class_name}: {n_files} gambar")



=== TRAIN ===
Daun: 1400 gambar
Kardus: 1400 gambar
Baterai: 1400 gambar
Sterofom: 1400 gambar
Logam: 1400 gambar
Masker: 1400 gambar
Lampu: 1400 gambar
Plastik: 1400 gambar
Elektronik: 1400 gambar
Pakaian: 1400 gambar
Sampah Makanan: 1400 gambar
Kaca: 1400 gambar
Kertas: 1400 gambar

=== VAL ===
Daun: 300 gambar
Kardus: 300 gambar
Baterai: 300 gambar
Sterofom: 300 gambar
Logam: 300 gambar
Masker: 300 gambar
Lampu: 300 gambar
Plastik: 300 gambar
Elektronik: 300 gambar
Pakaian: 300 gambar
Sampah Makanan: 300 gambar
Kaca: 300 gambar
Kertas: 300 gambar

=== TEST ===
Daun: 300 gambar
Kardus: 300 gambar
Baterai: 300 gambar
Sterofom: 300 gambar
Logam: 300 gambar
Masker: 300 gambar
Lampu: 300 gambar
Plastik: 300 gambar
Elektronik: 300 gambar
Pakaian: 300 gambar
Sampah Makanan: 300 gambar
Kaca: 300 gambar
Kertas: 300 gambar


# MODELING

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Augmentasi data untuk training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Hanya rescale untuk validasi
val_datagen = ImageDataGenerator(rescale=1./255)

# Buat data generator
train_generator = train_datagen.flow_from_directory(
    directory=f'{base_dir}/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    directory=f'{base_dir}/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 18200 images belonging to 13 classes.
Found 3900 images belonging to 13 classes.


In [ ]:
print(train_generator.class_indices)

{'Baterai': 0, 'Daun': 1, 'Elektronik': 2, 'Kaca': 3, 'Kardus': 4, 'Kertas': 5, 'Lampu': 6, 'Logam': 7, 'Masker': 8, 'Pakaian': 9, 'Plastik': 10, 'Sampah Makanan': 11, 'Sterofom': 12}


### MODEL 1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Model CNN sederhana
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callback
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

# Training model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
569/569 ━━━━━━━━━━━━━━━━━━━━ 232s 395ms/step - accuracy: 0.2322 - loss: 2.3242 - val_accuracy: 0.4100 - val_loss: 1.7445 - learning_rate: 0.0010
Epoch 2/20
569/569 ━━━━━━━━━━━━━━━━━━━━ 214s 375ms/step - accuracy: 0.3831 - loss: 1.8447 - val_accuracy: 0.4651 - val_loss: 1.5977 - learning_rate: 0.0010
Epoch 3/20
569/569 ━━━━━━━━━━━━━━━━━━━━ 218s 382ms/step - accuracy: 0.4265 - loss: 1.7177 - val_accuracy: 0.4967 - val_loss: 1.5136 - learning_rate: 0.0010
Epoch 4/20
569/569 ━━━━━━━━━━━━━━━━━━━━ 211s 370ms/step - accuracy: 0.4658 - loss: 1.6263 - val_accuracy: 0.5203 - val_loss: 1.4708 - learning_rate: 0.0010
Epoch 5/20
569/569 ━━━━━━━━━━━━━━━━━━━━ 210s 369ms/step - accuracy: 0.4772 - loss: 1.5798 - val_accuracy: 0.5118 - val_loss: 1.4761 - learning_rate: 0.0010
Epoch 6/20
569/569 ━━━━━━━━━━━━━━━━━━━━ 209s 367ms/step - accuracy: 0.5046 - loss: 1.5309 - val_accuracy: 0.5313 - val_loss: 1.4366 - learning_rate: 0.0010
Epoch 7/20
569/569 ━━━━━━━━━━━━━━━━━━━━ 211s 370ms/step - accura

In [ ]:
# Evaluasi pakai test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory=f'{base_dir}/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.2%}")

Found 3900 images belonging to 13 classes.
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.5667 - loss: 1.3087
Test Accuracy: 61.49%
